In [1]:
# Import our dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import datetime
import re
import csv
import json
import os


In [2]:
# Results
# IMDB
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 70529 entries, 0 to 70528
# Data columns (total 22 columns):
#  #   Column                 Non-Null Count  Dtype  
# ---  ------                 --------------  -----  
#  0   imdb_title_id          70529 non-null  object    recode imdb_id
#  1   title                  70529 non-null  object 
#  2   year                   70529 non-null  float64   1969+
#  3   genre                  70529 non-null  object    list (and individual indicators separate file)
#  4   duration               70529 non-null  int64  
#  5   country                70465 non-null  object     
#  6   language               69913 non-null  object 
#  7   director               70445 non-null  object    string to decod
#  8   writer                 69052 non-null  object    string to decod
#  9   production_company     66300 non-null  object    string to decod
#  10  actors                 70466 non-null  object    string to decode
#  11  avg_vote               70529 non-null  float64   newer than TMDB
#  12  votes                  70529 non-null  int64     newer than TMDB
#  13  budget                 21107 non-null  object 
#  14  usa_gross_income       14632 non-null  object    
#  15  worlwide_gross_income  30375 non-null  object 
#  16  metascore              12858 non-null  float64
#  17  reviews_from_users     63624 non-null  float64   # reviews 
#  18  reviews_from_critics   60360 non-null  float64   # reviews
#  19  release_year           70529 non-null  float64   # 1969+
#  20  release_year.1         70529 non-null  float64   drop
#  21  genre.1                70529 non-null  object    drop
# dtypes: float64(7), int64(2), object(13)  

In [3]:
# TMDB
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 37087 entries, 0 to 37086
# Data columns (total 19 columns):
#  #   Column                Non-Null Count  Dtype  
# ---  ------                --------------  -----  
#  0   budget                37087 non-null  int64     missing
#  1   genres                37087 non-null  object    drop - decode
#  2   id                    37087 non-null  int64     drop
#  3   imdb_id               37073 non-null  object    proprietary score
#  4   popularity            37087 non-null  float64
#  5   production_companies  37087 non-null  object    decode
#  6   production_countries  37087 non-null  object    decode
#  7   revenue               37087 non-null  float64
#  8   runtime               36871 non-null  float64
#  9   spoken_languages      37087 non-null  object    decode
#  10  vote_count            37087 non-null  float64   older than IMDB
#  11  release_year          37087 non-null  float64   older than IMDB
#  12  orig_lang_cd          37087 non-null  object    2 alpha code
#  13  collection            37087 non-null  object    0/1
#  14  website               37087 non-null  object    0/1
#  15  success               37087 non-null  object    0/1 based on average rating, so older than IMDB
#  16  0                     37087 non-null  int64     drop
#  17  genre_name            34969 non-null  object    list of max 3
#  18  0.1                   37087 non-null  int64     drop
#  19  release_year           70529 non-null  float64  
#  20  release_year.1         70529 non-null  float64  drop
#  21  genre.1                70529 non-null  object   drop

In [4]:
# Read in "The IMbd Movies dataset"
# if this is read in as csv, each row is a string and dictionaries are not recognized.

pd.set_option("max_columns", None)

IMDB_df = pd.read_csv("Resources/imdb_main.csv", low_memory=False)
IMDB_df.sort_index(axis=1, inplace=True)

IMDB_cols = IMDB_df.columns

IMDB_df.head()

,avg_vote,budget,country,director_list,duration,g_Action,g_Adult,g_Adventure,g_Animation,g_Biography,g_Comedy,g_Crime,g_Documentary,g_Drama,g_Family,g_Fantasy,g_History,g_Horror,g_Music,g_Musical,g_Mystery,g_News,g_Reality-TV,g_Romance,g_Sci-Fi,g_Sport,g_Thriller,g_War,g_Western,genre_list,imdb_id,language,metascore,release_year,reviews_from_critics,reviews_from_users,title,usa_gross_income,votes,worlwide_gross_income,year
0,6.3,NaN,Australia,['Raymond Longford'],106,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,"['Drama', ' Romance']",tt0010680,English,NaN,2005.0,10.0,11.0,The Sentimental Bloke,NaN,237,NaN,1919.0
1,6.8,NaN,Germany,['Arthur Robison'],90,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,"['Drama', ' Fantasy', ' Horror']",tt0013579,German,NaN,2016.0,16.0,15.0,Ombre ammonitrici,NaN,842,NaN,1923.0
2,7.0,NaN,France,['Jean Epstein'],45,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,"['Drama', ' Romance']",tt0017938,"None, French",NaN,1983.0,4.0,7.0,La glace à trois faces,NaN,759,NaN,1983.0
3,7.2,$ 2500000,USA,['Cecil B. DeMille'],160,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"['Biography', ' Drama', ' History']",tt0018054,English,NaN,2004.0,24.0,48.0,Il re dei re,NaN,1890,NaN,1927.0
4,8.1,NaN,USA,"['Edward Sedgwick', ' Buster Keaton']",76,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"['Comedy', ' Drama', ' Family']",tt0018742,English,NaN,1970.0,49.0,61.0,Il cameraman,NaN,10101,$ 1737460,1928.0


In [5]:
print(IMDB_df.info())
print(IMDB_df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   avg_vote               70529 non-null  float64
 1   budget                 21107 non-null  object 
 2   country                70465 non-null  object 
 3   director_list          70445 non-null  object 
 4   duration               70529 non-null  int64  
 5   g_Action               70529 non-null  int64  
 6   g_Adult                70529 non-null  int64  
 7   g_Adventure            70529 non-null  int64  
 8   g_Animation            70529 non-null  int64  
 9   g_Biography            70529 non-null  int64  
 10  g_Comedy               70529 non-null  int64  
 11  g_Crime                70529 non-null  int64  
 12  g_Documentary          70529 non-null  int64  
 13  g_Drama                70529 non-null  int64  
 14  g_Family               70529 non-null  int64  
 15  g_

In [6]:
# Read in the movies database
TMDB_df=pd.read_csv("Resources/tmdb_main.csv", low_memory=False)
TMDB_df.sort_index(axis=1, inplace=True)

TMDB_cols = TMDB_df.columns

print(TMDB_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37050 entries, 0 to 37049
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             37050 non-null  int64  
 1   collection         37050 non-null  object 
 2   g_Action           37050 non-null  int64  
 3   g_Adventure        37050 non-null  int64  
 4   g_Animation        37050 non-null  int64  
 5   g_Comedy           37050 non-null  int64  
 6   g_Crime            37050 non-null  int64  
 7   g_Documentary      37050 non-null  int64  
 8   g_Drama            37050 non-null  int64  
 9   g_Family           37050 non-null  int64  
 10  g_Fantasy          37050 non-null  int64  
 11  g_Foreign          37050 non-null  int64  
 12  g_History          37050 non-null  int64  
 13  g_Horror           37050 non-null  int64  
 14  g_Music            37050 non-null  int64  
 15  g_Mystery          37050 non-null  int64  
 16  g_Romance          370

In [7]:
TMDB_df.head()

,budget,collection,g_Action,g_Adventure,g_Animation,g_Comedy,g_Crime,g_Documentary,g_Drama,g_Family,g_Fantasy,g_Foreign,g_History,g_Horror,g_Music,g_Mystery,g_Romance,g_Thriller,g_War,g_Western,genre_name,imdb_id,orig_lang_cd,original_language,popularity,release_year,revenue,runtime,website
0,30000000,yes,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"['Animation', 'Comedy', 'Family']",tt0114709,en,en,21.946943,1995.0,373554033.0,81.0,yes
1,65000000,no,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,"['Adventure', 'Fantasy', 'Family']",tt0113497,en,en,17.015539,1995.0,262797249.0,104.0,no
2,0,yes,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,"['Romance', 'Comedy']",tt0113228,en,en,11.712900,1995.0,0.0,101.0,no
3,16000000,no,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,"['Comedy', 'Drama', 'Romance']",tt0114885,en,en,3.859495,1995.0,81452156.0,127.0,no
4,0,yes,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,['Comedy'],tt0113041,en,en,8.387519,1995.0,76578911.0,106.0,no


In [8]:
print(f'movies in IMDB: {len(IMDB_df)}')
print(f'\n movies in TMDB: {len(TMDB_df)}')

movies in IMDB: 70529

 movies in TMDB: 37050


In [9]:
merge_movies_df = pd.merge(IMDB_df, TMDB_df, on='imdb_id', how="outer", suffixes=['_IM','_TM'], indicator=True, validate="1:1")
print(merge_movies_df.describe())

           avg_vote      duration   g_Action_IM       g_Adult  g_Adventure_IM  \
count  70529.000000  70529.000000  70529.000000  70529.000000    70529.000000   
mean       5.790186    102.113414      0.161976      0.000028        0.078379   
std        1.275937     21.708421      0.368431      0.005325        0.268769   
min        1.000000     41.000000      0.000000      0.000000        0.000000   
25%        5.000000     90.000000      0.000000      0.000000        0.000000   
50%        6.000000     97.000000      0.000000      0.000000        0.000000   
75%        6.700000    110.000000      0.000000      0.000000        0.000000   
max        9.900000    808.000000      1.000000      1.000000        1.000000   

       g_Animation_IM   g_Biography   g_Comedy_IM    g_Crime_IM  \
count    70529.000000  70529.000000  70529.000000  70529.000000   
mean         0.028995      0.028045      0.345560      0.121751   
std          0.167794      0.165103      0.475554      0.327001   
mi

In [10]:
merge_movies_df.info()
merge_movies_df.sort_index(axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80604 entries, 0 to 80603
Data columns (total 70 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   avg_vote               70529 non-null  float64 
 1   budget_IM              21107 non-null  object  
 2   country                70465 non-null  object  
 3   director_list          70445 non-null  object  
 4   duration               70529 non-null  float64 
 5   g_Action_IM            70529 non-null  float64 
 6   g_Adult                70529 non-null  float64 
 7   g_Adventure_IM         70529 non-null  float64 
 8   g_Animation_IM         70529 non-null  float64 
 9   g_Biography            70529 non-null  float64 
 10  g_Comedy_IM            70529 non-null  float64 
 11  g_Crime_IM             70529 non-null  float64 
 12  g_Documentary_IM       70529 non-null  float64 
 13  g_Drama_IM             70529 non-null  float64 
 14  g_Family_IM            70529 non-null 

In [11]:
# note: runtime same as duration
# drop _merge, g_Adult


In [12]:
# merge and get counts on matches
temp=merge_movies_df._merge
temp.value_counts(dropna=False)


left_only     43554
both          26975
right_only    10075
Name: _merge, dtype: int64

In [13]:
# keep just the indices for left and both - master match list
cond1 = merge_movies_df._merge == "left_only" 
cond2 = merge_movies_df._merge == "both" 
mask = cond1 | cond2
merge_movies_df = merge_movies_df[mask]
merge_indices = merge_movies_df["imdb_id"]

In [14]:
# Save to csv
print(merge_movies_df.info)
file_path ="Resources/merged_movies.csv"
merge_movies_df.to_csv(file_path,index=False)

<bound method DataFrame.info of           _merge  avg_vote  budget_IM  budget_TM collection          country  \
0      left_only       6.3        NaN        NaN        NaN        Australia   
1      left_only       6.8        NaN        NaN        NaN          Germany   
2      left_only       7.0        NaN        NaN        NaN           France   
3      left_only       7.2  $ 2500000        NaN        NaN              USA   
4      left_only       8.1        NaN        NaN        NaN              USA   
...          ...       ...        ...        ...        ...              ...   
70524  left_only       5.3        NaN        NaN        NaN  France, Belgium   
70525  left_only       7.7        NaN        NaN        NaN      Netherlands   
70526  left_only       7.9        NaN        NaN        NaN            India   
70527  left_only       6.4        NaN        NaN        NaN           Turkey   
70528  left_only       6.7        NaN        NaN        NaN            Spain   

       

In [15]:
print(len(merge_indices))
merge_indices

70529


0        tt0010680
1        tt0013579
2        tt0017938
3        tt0018054
4        tt0018742
           ...    
70524    tt9908390
70525    tt9911196
70526    tt9911774
70527    tt9914286
70528    tt9914942
Name: imdb_id, Length: 70529, dtype: object

In [16]:
# keep just the indices for match = both for comparison
cond2 = merge_movies_df._merge == "both" 
mask = cond2
compare_indices = merge_movies_df[mask]["imdb_id"].sort_values()

In [17]:
print(len(compare_indices))
print(type(compare_indices))
print(compare_indices.axes)
print(compare_indices.name)
compare_indices


26975
<class 'pandas.core.series.Series'>
[Int64Index([   11,    27,    74,    81,   119,   151,   157,   164,   165,
              168,
            ...
            65562, 65568, 65662, 65670, 65708, 65744, 65757, 65774, 65851,
            65967],
           dtype='int64', length=26975)]
imdb_id


11       tt0035423
27       tt0054724
74       tt0061592
81       tt0062181
119      tt0063142
           ...    
65744    tt6712026
65757    tt6721848
65774    tt6731636
65851    tt6772874
65967    tt6830780
Name: imdb_id, Length: 26975, dtype: object

In [18]:
merge_movies_df.columns

Index(['_merge', 'avg_vote', 'budget_IM', 'budget_TM', 'collection', 'country',
       'director_list', 'duration', 'g_Action_IM', 'g_Action_TM', 'g_Adult',
       'g_Adventure_IM', 'g_Adventure_TM', 'g_Animation_IM', 'g_Animation_TM',
       'g_Biography', 'g_Comedy_IM', 'g_Comedy_TM', 'g_Crime_IM', 'g_Crime_TM',
       'g_Documentary_IM', 'g_Documentary_TM', 'g_Drama_IM', 'g_Drama_TM',
       'g_Family_IM', 'g_Family_TM', 'g_Fantasy_IM', 'g_Fantasy_TM',
       'g_Foreign', 'g_History_IM', 'g_History_TM', 'g_Horror_IM',
       'g_Horror_TM', 'g_Music_IM', 'g_Music_TM', 'g_Musical', 'g_Mystery_IM',
       'g_Mystery_TM', 'g_News', 'g_Reality-TV', 'g_Romance_IM',
       'g_Romance_TM', 'g_Sci-Fi', 'g_Sport', 'g_Thriller_IM', 'g_Thriller_TM',
       'g_War_IM', 'g_War_TM', 'g_Western_IM', 'g_Western_TM', 'genre_list',
       'genre_name', 'imdb_id', 'language', 'metascore', 'orig_lang_cd',
       'original_language', 'popularity', 'release_year_IM', 'release_year_TM',
       'revenue', '

In [19]:
# compare values on matches
# genres, note: IMDB has some genres TMDB does not have. Drop to compare

print(f'\n IMDB cols:  {IMDB_cols}')
print(f'\n TMDB cols:  {TMDB_cols}')

#      g_Action_IM            70529 non-null  float64 
#  9   g_Action_TM            26975 non-null  float64 
#  11  g_Adventure_IM         70529 non-null  float64 
#  12  g_Adventure_TM         26975 non-null  float64 
#  13  g_Animation_IM         70529 non-null  float64 
#  14  g_Animation_TM         26975 non-null  float64 
#  16  g_Comedy_IM            70529 non-null  float64 
#  17  g_Comedy_TM            26975 non-null  float64 
#  18  g_Crime_IM             70529 non-null  float64 
#  19  g_Crime_TM             26975 non-null  float64 
#  20  g_Documentary_IM       70529 non-null  float64 
#  21  g_Documentary_TM       26975 non-null  float64 
#  22  g_Drama_IM             70529 non-null  float64 
#  23  g_Drama_TM             26975 non-null  float64 
#  24  g_Family_IM            70529 non-null  float64 
#  25  g_Family_TM            26975 non-null  float64 
#  26  g_Fantasy_IM           70529 non-null  float64 
#  27  g_Fantasy_TM           26975 non-null  float64 
#  29  g_History_IM           70529 non-null  float64 
#  30  g_History_TM           26975 non-null  float64 
#  31  g_Horror_IM            70529 non-null  float64 
#  32  g_Horror_TM            26975 non-null  float64 
#  33  g_Music_IM             70529 non-null  float64 
#  34  g_Music_TM             26975 non-null  float64 
#  36  g_Mystery_IM           70529 non-null  float64 
#  37  g_Mystery_TM           26975 non-null  float64 
#  40  g_Romance_IM           70529 non-null  float64 
#  41  g_Romance_TM           26975 non-null  float64 
#  44  g_Thriller_IM          70529 non-null  float64 
#  45  g_Thriller_TM          26975 non-null  float64 
#  46  g_War_IM               70529 non-null  float64 
#  47  g_War_TM               26975 non-null  float64 
#  48  g_Western_IM           70529 non-null  float64 
#  49  g_Western_TM 
# save list of cols to compare in pd.DataFrame.compare
compare_cols = ['imdb_id','g_Action', 'g_Adventure', 'g_Animation', 'g_Comedy', 'g_Crime', 'g_Documentary', 'g_Drama', 'g_Family',
       'g_Fantasy', 'g_History', 'g_Horror', 'g_Music', 'g_Mystery', 'g_Romance', 'g_Thriller','g_War', 'g_Western']


 IMDB cols:  Index(['avg_vote', 'budget', 'country', 'director_list', 'duration',
       'g_Action', 'g_Adult', 'g_Adventure', 'g_Animation', 'g_Biography',
       'g_Comedy', 'g_Crime', 'g_Documentary', 'g_Drama', 'g_Family',
       'g_Fantasy', 'g_History', 'g_Horror', 'g_Music', 'g_Musical',
       'g_Mystery', 'g_News', 'g_Reality-TV', 'g_Romance', 'g_Sci-Fi',
       'g_Sport', 'g_Thriller', 'g_War', 'g_Western', 'genre_list', 'imdb_id',
       'language', 'metascore', 'release_year', 'reviews_from_critics',
       'reviews_from_users', 'title', 'usa_gross_income', 'votes',
       'worlwide_gross_income', 'year'],
      dtype='object')

 TMDB cols:  Index(['budget', 'collection', 'g_Action', 'g_Adventure', 'g_Animation',
       'g_Comedy', 'g_Crime', 'g_Documentary', 'g_Drama', 'g_Family',
       'g_Fantasy', 'g_Foreign', 'g_History', 'g_Horror', 'g_Music',
       'g_Mystery', 'g_Romance', 'g_Thriller', 'g_War', 'g_Western',
       'genre_name', 'imdb_id', 'orig_lang_cd', 'origina

In [20]:
TMDB_compare = TMDB_df[compare_cols].copy().sort_values("imdb_id", axis=0)
print(TMDB_compare.head())
TMDB_compare = pd.merge(TMDB_compare, compare_indices, how="inner", on="imdb_id")

print(len(TMDB_compare))
TMDB_compare.head()

         imdb_id  g_Action  g_Adventure  g_Animation  g_Comedy  g_Crime  \
4158   tt0035423         0            0            0         1        0   
31803  tt0042872         0            0            0         0        0   
36053  tt0054724         0            0            0         0        0   
31268  tt0061592         0            0            0         0        0   
32019  tt0062181         0            0            0         1        0   

       g_Documentary  g_Drama  g_Family  g_Fantasy  g_History  g_Horror  \
4158               0        0         0          1          0         0   
31803              0        1         0          0          0         0   
36053              0        0         0          0          0         1   
31268              0        0         0          0          0         0   
32019              0        1         0          0          0         0   

       g_Music  g_Mystery  g_Romance  g_Thriller  g_War  g_Western  
4158         0          0    

,imdb_id,g_Action,g_Adventure,g_Animation,g_Comedy,g_Crime,g_Documentary,g_Drama,g_Family,g_Fantasy,g_History,g_Horror,g_Music,g_Mystery,g_Romance,g_Thriller,g_War,g_Western
0,tt0035423,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
1,tt0054724,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,tt0061592,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0062181,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,tt0063142,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0


In [21]:
# compare on genres
IMDB_compare = IMDB_df[compare_cols].copy().sort_values("imdb_id", axis=0)
print(IMDB_compare.head())
IMDB_compare = pd.merge(IMDB_compare, compare_indices, how="inner", on="imdb_id")

print(len(IMDB_compare))
IMDB_compare.head()

     imdb_id  g_Action  g_Adventure  g_Animation  g_Comedy  g_Crime  \
0  tt0010680         0            0            0         0        0   
1  tt0013579         0            0            0         0        0   
2  tt0017938         0            0            0         0        0   
3  tt0018054         0            0            0         0        0   
4  tt0018742         0            0            0         1        0   

   g_Documentary  g_Drama  g_Family  g_Fantasy  g_History  g_Horror  g_Music  \
0              0        1         0          0          0         0        0   
1              0        1         0          1          0         1        0   
2              0        1         0          0          0         0        0   
3              0        1         0          0          1         0        0   
4              0        1         1          0          0         0        0   

   g_Mystery  g_Romance  g_Thriller  g_War  g_Western  
0          0          1           0 

,imdb_id,g_Action,g_Adventure,g_Animation,g_Comedy,g_Crime,g_Documentary,g_Drama,g_Family,g_Fantasy,g_History,g_Horror,g_Music,g_Mystery,g_Romance,g_Thriller,g_War,g_Western
0,tt0035423,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
1,tt0054724,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,tt0061592,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0062181,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,tt0063142,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0


In [22]:
print(IMDB_compare.shape)
print(TMDB_compare.shape)

(26975, 18)
(26975, 18)


In [23]:
# compare genre values, differences are kept, matches =NaN
diff = IMDB_compare.compare(TMDB_compare, align_axis=1, keep_shape=False, keep_equal=False)
diff.head()

g_Action       g_Adventure       g_Animation       g_Comedy       g_Crime  \
       self other        self other        self other     self other    self   
3       NaN   NaN         NaN   NaN         NaN   NaN      0.0   1.0     NaN   
8       NaN   NaN         NaN   NaN         NaN   NaN      NaN   NaN     NaN   
12      0.0   1.0         NaN   NaN         NaN   NaN      NaN   NaN     NaN   
13      NaN   NaN         NaN   NaN         NaN   NaN      NaN   NaN     1.0   
14      NaN   NaN         NaN   NaN         NaN   NaN      NaN   NaN     NaN   

         g_Documentary       g_Drama       g_Family       g_Fantasy        \
   other          self other    self other     self other      self other   
3    NaN           NaN   NaN     NaN   NaN      NaN   NaN       NaN   NaN   
8    NaN           NaN   NaN     NaN   NaN      NaN   NaN       NaN   NaN   
12   NaN           NaN   NaN     NaN   NaN      NaN   NaN       NaN   NaN   
13   0.0           NaN   NaN     0.0   1.0      NaN   NaN       NaN   NaN   
14   NaN           NaN   NaN     NaN   NaN      NaN   NaN       NaN   NaN   

   g_History       g_Horror       g_Music       g_Mystery       g_Romance  \
        self other     self other    self other      self other      self   
3        NaN   NaN      NaN   NaN     NaN   NaN       NaN   NaN       NaN   
8        NaN   NaN      NaN   NaN     NaN   NaN       NaN   NaN       1.0   
12       NaN   NaN      1.0   0.0     NaN   NaN       0.0   1.0       NaN   
13       NaN   NaN      NaN   NaN     NaN   NaN       NaN   NaN       NaN   
14       NaN   NaN      NaN   NaN     NaN   NaN       NaN   NaN       1.0   

         g_Thriller       g_War       g_Western        
   other       self other  self other      self other  
3    NaN        NaN   NaN   NaN   NaN       NaN   NaN  
8    0.0        NaN   NaN   NaN   NaN       NaN   NaN  
12   NaN        0.0   1.0   NaN   NaN       NaN   NaN  
13   NaN        NaN   NaN   NaN   NaN       NaN   NaN  
14   0.0        NaN   NaN   NaN   NaN       NaN   NaN

In [24]:
# count non-NaN cells = number of mismatches by column
diff.count(axis=0)

g_Action       self     2017
               other    2017
g_Adventure    self     1607
               other    1607
g_Animation    self       46
               other      46
g_Comedy       self     1991
               other    1991
g_Crime        self     2035
               other    2035
g_Documentary  self       89
               other      89
g_Drama        self     4004
               other    4004
g_Family       self     1035
               other    1035
g_Fantasy      self     1210
               other    1210
g_History      self      640
               other     640
g_Horror       self      762
               other     762
g_Music        self      564
               other     564
g_Mystery      self     1327
               other    1327
g_Romance      self     2796
               other    2796
g_Thriller     self     3554
               other    3554
g_War          self      446
               other     446
g_Western      self      134
               other     134
dtype: int64

In [25]:
# count non-NaN cells = number of mismatches by row
diff.count(axis=1)

3        2
8        2
12       8
13       4
14       2
        ..
26968    2
26969    2
26970    4
26972    2
26973    2
Length: 14762, dtype: int64